# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

[Wine Quality Data Set](https://archive.ics.uci.edu/ml/datasets/Wine+Quality)

In [1]:
from azureml.core.workspace import Workspace
from azureml.core.datastore import Datastore
from azureml.core.compute import ComputeTarget
from azureml.core.compute.amlcompute import AmlCompute
from azureml.exceptions import ComputeTargetException
from azureml.core.experiment import Experiment
from azureml.core.run import Run
from azureml.core.dataset import Dataset
from azureml.core.model import Model

from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice


from azureml.core.webservice import Webservice
from azureml.core.authentication import InteractiveLoginAuthentication

import pandas as pd

from azureml.pipeline.core.pipeline import Pipeline
from azureml.pipeline.core import PipelineData
from azureml.pipeline.core import TrainingOutput
from azureml.pipeline.core.run import PipelineRun
from azureml.pipeline.steps.automl_step import AutoMLStep

from azureml.train.automl.automlconfig import AutoMLConfig
from azureml.data import TabularDataset
from azureml.widgets.run_details import RunDetails

from azureml.automl.core.shared import constants

import json
import pickle
import requests

from pprint import pprint

import logging
import joblib

from train import clean_data, get_dataset
import capstone_constants as c_constants



## Dataset

### Overview
Overview¶
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

This machine learning program detects the wine quality of white wine.
The task is to determine if the wine quality is "good'" (1) or "not good" (0).
More information about the dataset is provided in the README for this Capstone Project.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

The dataset is external and the URI as defined in capstone_constants.py is:
TABULAR_WINE_DATA_URI = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv'

Note that the oriignal data above qualifies quality as a classfification between 1 and 10. However, this Capstone project transforms quality > 7 as "good" (1) and otherwise "not good" (0). The project is thus framed as a binary classification challenge.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment=Experiment(ws, c_constants.AUTOML_EXPERIMENT_NAME)

In [3]:
# Next, let's use if it exists, or create if required, a compute cluster to be used by the ML

# Access the compute cluster. If it exists, we will have the compute object. 
# If it does not exist, an exception will be thrown upon which the compute cluster is created
try:
    cc = ComputeTarget(workspace=ws, name=c_constants.COMPUTE_CLUSTER_AUTOML)
    print(f'Compute Cluster target exists and we have a handle to the same')
except ComputeTargetException:
    # Failed to obtain the compute cluster object
    # In all likelihood, a compute cluster of that name has not been created
    # Attempt to create the compute cluster
    # First set up the configuration

    # Specify the configuration of the compute cluster
    cc_cfg = AmlCompute.provisioning_configuration(vm_size='Standard_DS12_v2', min_nodes=1, max_nodes=6)
    cc = ComputeTarget.create(workspace=ws, name=c_constants.COMPUTE_CLUSTER_AUTOML, provisioning_configuration=cc_cfg)

# At this point - we have access to the compute cluster object. Wait for the compute target to complete provisioing
cc.wait_for_completion(show_output='True')

Compute Cluster target exists and we have a handle to the same
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
# grab the data and create a dataset
train_ds = get_dataset(ws)

# Take a peek at the data by converting the same to a Pandas dataframe
proj_df = train_ds.to_pandas_dataframe()

# print the data
proj_df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,1.712604,-1.371514,0.213258,0.474873,-0.904997,-1.311690,-1.843864,0.525802,-0.385870,-1.225374,0.435336,0
1,0.172079,-0.577820,-0.613053,-1.043248,-0.401518,-0.900098,-1.585029,-1.279671,-0.584545,-0.962507,0.963524,0
2,0.764589,0.215874,3.105347,-1.003816,5.228296,0.511075,0.250349,0.258324,-0.716995,-0.612016,-1.311750,0
3,-0.183426,0.017450,0.047996,0.494589,0.101961,-0.194512,-0.643809,0.174737,0.408829,0.439455,0.394706,1
4,-0.657434,-0.081762,-1.274101,0.257999,0.101961,0.569873,1.120977,0.726410,0.276379,-0.173903,-1.230491,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3913,1.120095,-0.875455,-0.117266,0.198851,-1.408477,0.687471,-0.337913,-0.417056,-1.313019,-0.874884,0.394706,0
3914,-0.183426,0.315085,-1.274101,0.257999,-0.126893,1.275460,1.450404,0.545862,-0.120971,-0.436771,-0.905451,0
3915,-0.538932,-1.272302,-0.365159,-0.905237,-0.081122,-0.841299,-0.549688,-0.791525,-0.584545,-0.962507,-0.255373,0
3916,0.527585,-0.379397,0.626414,1.421235,0.193503,0.334678,0.932733,1.595711,1.203527,-0.524394,-0.661672,0


In [5]:
proj_df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000
mean,0.006576,0.009056,-0.002346,0.006893,0.012078,0.013468,0.010146,0.006848,-0.010731,-0.002125,-0.005237,0.204696
std,1.009986,1.007306,1.009177,1.003865,1.033014,1.013921,1.007248,1.007763,0.988684,0.999605,0.994886,0.403531
min,-3.619982,-1.966784,-2.761461,-1.141827,-1.500018,-1.958477,-3.043919,-2.306115,-3.101091,-2.364468,-2.043089,0.000000
25%,-0.657434,-0.677032,-0.613053,-0.924953,-0.447289,-0.723701,-0.690870,-0.761433,-0.650770,-0.699639,-0.824192,0.000000
50%,-0.064924,-0.180973,-0.117266,-0.215182,-0.126893,-0.076914,-0.079078,-0.076023,-0.054746,-0.086281,-0.092853,0.000000
75%,0.527585,0.414297,0.461152,0.691748,0.193503,0.628672,0.697428,0.699662,0.607503,0.527077,0.719745,0.000000
max,8.704217,8.152811,10.955302,11.712916,13.741673,14.916791,7.097720,15.029763,4.183648,5.171074,2.995020,1.000000


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

This project is a classification issue. More so, it is a binary classification issue as teh outcome is whether the wine is of a good quality or not.

AUC_weighted is an apporpriate metric to target for a binary classification.
[Set up AutoML training with Python](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train)

It is generally recommended to enable early stopping as it is possible that after a while no further improvement in the model is feasible.

There is enrally limited to no benefit to using a large number of cross validations. In this instance, we have set it to 3.

In [6]:
# TODO: Put your automl settings here

automl_settings = {
    "iterations" : 20,
    "experiment_timeout_minutes" : 30,
    "enable_early_stopping" : True,
    "iteration_timeout_minutes" : 5,
    "max_concurrent_iterations" : 5,
    "max_cores_per_iteration" : -1,
    "n_cross_validations" : 3,
    "primary_metric" : 'AUC_weighted',
    "verbosity" : logging.INFO,
}

# Provide the remainder of the settings/configuration
# Note that we are not providing a validation data set - and we may need to
# 


# TODO: Put your automl config here
automl_config = AutoMLConfig(
    compute_target = cc,
    task='classification',
    training_data=train_ds,
    label_column_name=c_constants.LABEL_COLUMN_NAME,
    featurization='auto',
    model_explainability=True,
    debug_log=c_constants.DEBUG_LOG,
    **automl_settings)

In [7]:
# TODO: Submit your experiment
automl_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
exp-capstone-automl,AutoML_f180290c-756b-4a06-82d2-d40dfe4f15f5,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [9]:
automl_run.wait_for_completion()

{'runId': 'AutoML_f180290c-756b-4a06-82d2-d40dfe4f15f5',
 'target': 'CPU-CC-AUTOML',
 'status': 'Completed',
 'startTimeUtc': '2021-10-24T23:15:41.722188Z',
 'endTimeUtc': '2021-10-24T23:23:42.908123Z',
 'services': {},
 'properties': {'num_iterations': '20',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'CPU-CC-AUTOML',
  'AMLSettingsJsonString': '{"path":null,"name":"exp-capstone-automl","subscription_id":"d4ad7261-832d-46b2-b093-22156001df5b","resource_group":"aml-quickstarts-161910","workspace_name":"quick-starts-ws-161910","region":"southcentralus","compute_target":"CPU-CC-AUTOML","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":20,"

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [10]:
def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0]+ ' - ')
        elif hasattr(step[1], '_base_learners') and hasattr(step[1], '_meta_learner'):
            print("\nMeta Learner")
            pprint(step[1]._meta_learner)
            print()
            for estimator in step[1]._base_learners:
                print_model(estimator[1], estimator[0]+ ' - ')
        else:
            pprint(step[1].get_params())
            print()

In [11]:
automl_best_run, automl_best_model = automl_run.get_output()

automl_best_run_metrics = automl_best_run.get_metrics()

print(f'********** Best AutoML accuracy: {automl_best_run_metrics.get("accuracy")}')
print(f'********** printing Best AutoML run:\n{automl_best_run}\n\nPrinting model:')

print_model(automl_best_model)

********** Best AutoML accuracy: 0.866513527309852
********** printing Best AutoML run:
Run(Experiment: exp-capstone-automl,
Id: AutoML_f180290c-756b-4a06-82d2-d40dfe4f15f5_18,
Type: azureml.scriptrun,
Status: Completed)

Printing model:
datatransformer
{'enable_dnn': False,
 'enable_feature_sweeping': True,
 'feature_sweeping_config': {},
 'feature_sweeping_timeout': 86400,
 'featurization_config': None,
 'force_text_dnn': False,
 'is_cross_validation': True,
 'is_onnx_compatible': False,
 'observer': None,
 'task': 'classification',
 'working_dir': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook161910/code/Users/odl_user_161910'}

prefittedsoftvotingclassifier
{'estimators': ['5', '8', '13', '0', '12', '1', '7', '11'],
 'weights': [0.26666666666666666,
             0.06666666666666667,
             0.2,
             0.06666666666666667,
             0.2,
             0.06666666666666667,
             0.06666666666666667,
             0.06666666666666667]}

5 - standardscale

In [12]:
print(automl_run.get_metrics())

{'experiment_status': ['DatasetEvaluation', 'FeaturesGeneration', 'DatasetFeaturization', 'DatasetFeaturizationCompleted', 'DatasetCrossValidationSplit', 'ModelSelection'], 'experiment_status_description': ['Gathering dataset statistics.', 'Generating features for the dataset.', 'Beginning to fit featurizers and featurize the dataset.', 'Completed fit featurizers and featurizing the dataset.', 'Generating individually featurized CV splits.', 'Beginning model selection.'], 'AUC_weighted': 0.9008417094606811, 'average_precision_score_micro': 0.9445614606320567, 'recall_score_micro': 0.866513527309852, 'precision_score_micro': 0.866513527309852, 'norm_macro_recall': 0.47015201030275455, 'recall_score_macro': 0.7350760051513773, 'matthews_correlation': 0.5534033792044833, 'f1_score_weighted': 0.8556140915060618, 'f1_score_macro': 0.7630065131819332, 'average_precision_score_macro': 0.8624638144067115, 'weighted_accuracy': 0.9304711252342154, 'precision_score_macro': 0.8285915911940213, 'AU

In [13]:
#TODO: Save the best model
joblib.dump(automl_best_model, c_constants.DEPLOYED_AUTOML_MODEL_PATH)

['outputs/best_automl.pkl']

In [14]:
automl_best_run.download_file(constants.CONDA_ENV_FILE_PATH, c_constants.BEST_RUN_ENV)

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [15]:
# Refer - https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?tabs=python

# Tutorial: Deploy an image classification model in Azure Container Instances -
# https://docs.microsoft.com/en-us/azure/machine-learning/tutorial-deploy-models-with-aml

# Register the model
registered_model = automl_run.register_model(description=c_constants.DEPLOYED_AUTOML_MODEL_DESCRIPTION)
print(f'{automl_run.model_id}')
print(f'{registered_model.name}  {registered_model.id}  {registered_model.version}')


AutoMLf180290c718
AutoMLf180290c718  AutoMLf180290c718:1  1


In [17]:
curated_env = Environment.get(workspace=ws, name=c_constants.CURATED_ENV_NAME)

# Save the curated environment
curated_env.save_to_directory(path=c_constants.ENV_DIR, overwrite=True)


In [19]:
# Create an inference config

inference_config = InferenceConfig(
    environment=curated_env,
    source_directory=c_constants.INFERENCE_SOURCE_DIRECTORY,
    entry_script=c_constants.INFERENCE_SCORING_SCRIPT,
)

aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)


In [20]:

service = Model.deploy(workspace=ws,
                       name=c_constants.DEPLOYED_SERVICE,
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)



Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-10-24 23:43:09+00:00 Creating Container Registry if not exists..
2021-10-24 23:53:10+00:00 Registering the environment.
2021-10-24 23:53:10+00:00 Use the existing image.
2021-10-24 23:53:10+00:00 Generating deployment configuration.
2021-10-24 23:53:12+00:00 Submitting deployment to compute..
2021-10-24 23:53:15+00:00 Checking the status of deployment white-wine-service..
2021-10-24 23:59:43+00:00 Checking the status of inference endpoint white-wine-service.
Failed


ERROR:azureml.core.webservice.webservice:Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Operation ID: efe423b4-7942-44c6-ace0-29ed59c24d1a
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Error in entry script, ImportError: cannot import name 'ContextVar', please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, ImportError: cannot import name 'ContextVar', please run print(service.get_logs()) to get details."
    }
  ]
}



WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Operation ID: efe423b4-7942-44c6-ace0-29ed59c24d1a
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Error in entry script, ImportError: cannot import name 'ContextVar', please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, ImportError: cannot import name 'ContextVar', please run print(service.get_logs()) to get details."
    }
  ]
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current service state: Unhealthy\nOperation ID: efe423b4-7942-44c6-ace0-29ed59c24d1a\nMore information can be found using '.get_logs()'\nError:\n{\n  \"code\": \

In [22]:
logs = service.get_logs()

for line in logs.split('\n'):
    print(line)


2021-10-25T00:01:58,206970200+00:00 - gunicorn/run 
2021-10-25T00:01:58,215733100+00:00 - iot-server/run 
2021-10-25T00:01:58,216439300+00:00 - rsyslog/run 
/usr/sbin/nginx: /azureml-envs/azureml_2dc5fa74cf7028a6dad039c363b70698/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_2dc5fa74cf7028a6dad039c363b70698/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-10-25T00:01:58,260244800+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_2dc5fa74cf7028a6dad039c363b70698/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_2dc5fa74cf7028a6dad039c363b70698/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_2dc5fa74cf7028a6dad039c363b70698/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
# To enable ApplicationInsights on the service (webservice), 
# * first access the endpoint using the name assigned at the time of deployment
# * next update webservice parameters such as enabling application insights (enable_app_insights)

webservice = Webservice(
    workspace = ws,
    name=c_constants.DEPLOYED_SERVICE
)

webservice.update(
    enable_app_insights=True
)

# At this point application insights (logging is enabled) and can be
# checked in the GUI in AutoML studio

In [ ]:
# URL for the web service, should be similar to:
# 'http://8530a665-66f3-49c8-a953-b82a2d312917.eastus.azurecontainer.io/score'

# From the tail end of the code at
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?tabs=python
# - Deploy machine learning models to Azure


scoring_uri = webservice.scoring_uri

# If the service is authenticated, set the key or token
key, _ = webservice.get_keys()

# Set the appropriate headers
headers = {"Content-Type": "application/json"}
headers["Authorization"] = f"Bearer {key}"



# fixed ac	   volatile ac	citric acid	  residual sugar	chlorides	  free sulfurdi	total sulfurdi	density	       pH	        sulphates	    alcohol	quality		
# 0.883090875	0.3150853064	-0.5304215055	-0.1166025484	-0.447289012	-0.7237011554	-0.6908704601	-0.01249670459	1.004852702	0.4394546089	0.3947056997	0		
# 0.7645889612	1.307202455	-0.8609459206	1.657825186	0.3765862299	-0.4297069397	0.8386109571	1.655893566	-0.05474573919	0.001341709573	-0.6616718988	0		




# Two sets of data to score, so we get two results back
# data = {"data":
#         [
#           {
#             "fixed acidity": 0.883090875,
#             "volatile acidity": "0.3150853064",
#             "citric acid": "-0.5304215055",
#             "residual sugar": "-0.1166025484",
#             "chlorides": "-0.447289012",
#             "free sulfur dioxide": "-0.7237011554",
#             "total sulfur dioxide": "-0.6908704601",
#             "density": "-0.01249670459",
#             "pH": "1.004852702",
#             "sulphates": "0.4394546089",
#             "alcohol": 0.3947056997,
#           },
#           {
#             "fixed acidity": 0.7645889612,
#             "volatile acidity": "1.307202455",
#             "citric acid": "-0.8609459206",
#             "residual sugar": "1.657825186",
#             "chlorides": "0.3765862299",
#             "free sulfur dioxide": "-0.4297069397",
#             "total sulfur dioxide": "0.8386109571",
#             "density": "1.655893566",
#             "pH": "-0.05474573919",
#             "sulphates": "0.001341709573",
#             "alcohol": 0.3947056997,
#           },
#       ]
#     }


data = {"data":
        [
          [
           0.883090875,
           0.3150853064,
          -0.5304215055,
          -0.1166025484,
          -0.447289012,
          -0.7237011554,
          -0.6908704601,
          -0.01249670459,
          1.004852702,
          0.4394546089,
          0.3947056997
          ],
          [
          0.7645889612,
          1.307202455,
          -0.8609459206,
          1.657825186,
          0.3765862299,
          -0.4297069397,
          0.8386109571,
          1.655893566,
          -0.05474573919,
          0.001341709573,
          0.3947056997
          ]
        ]
    }

# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())

TODO: In the cell below, print the logs of the web service and delete the service

In [21]:
logs = webservice.get_logs()

for line in logs.split('\n'):
    print(line)



NameError: name 'webservice' is not defined

In [ ]:
# Clean up any resources
# Delete the Webservice
# delete the compute cluster

webservice.delete()
cc.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
